In [1]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [2]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [3]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [4]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [5]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [6]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [7]:
## 
n='Input_C_096' # 改名稱
##
dire= ['Data/BayesianRidge_Pre_0/','Data/BayesianRidge_Pre_1/','Data/BayesianRidge_Pre_2/','Data/BayesianRidge_Pre_3/','Data/BayesianRidge_Pre_4/','Data/BayesianRidge_Pre_5/','Data/BayesianRidge_Pre_6/','Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_C_096
Data/BayesianRidge_Pre_0/
feature number: 2
train_rmse: 0.006858067034926106
test_rmse: 0.006841852168658236
test_r2: 0.04405561336890632
feature number: 3
train_rmse: 0.006595013404591339
test_rmse: 0.006542715234996273
test_r2: 0.12489277294863346
feature number: 4
train_rmse: 0.00652400035076225
test_rmse: 0.006489824852675546
test_r2: 0.13935144913747918
feature number: 5
train_rmse: 0.006467192358257015
test_rmse: 0.00643263652181165
test_r2: 0.15449931462716368
feature number: 6
train_rmse: 0.006308642787004498
test_rmse: 0.006423143958042055
test_r2: 0.1564820027363833
feature number: 7
train_rmse: 0.006267577889390719
test_rmse: 0.006418183911259129
test_r2: 0.15741909113668434
feature number: 8
train_rmse: 0.0062566942671165564
test_rmse: 0.006409919093517805
test_r2: 0.15952062256287705
feature number: 9
train_rmse: 0.006252818244812069
test_rmse: 0.006426518435056481
test_r2: 0.15519405301366457
feature number: 10
train_rmse: 0.006244247713191054
test_rmse: 0.006

train_rmse: 0.00611892580874809
test_rmse: 0.006504258891569793
test_r2: 0.1349892245408853
feature number: 17
train_rmse: 0.006108416596479473
test_rmse: 0.006524010126384952
test_r2: 0.12959865354571548
feature number: 18
train_rmse: 0.006105170052562257
test_rmse: 0.006546193380112422
test_r2: 0.12374398857311734
feature number: 19
train_rmse: 0.006083421999626751
test_rmse: 0.0065485823298584555
test_r2: 0.12329773950060115
feature number: 20
train_rmse: 0.006029501410235101
test_rmse: 0.0064755927404061536
test_r2: 0.14232113085615822
Test_rmse_min = 1.fea_num: 12  2.rmse: 0.006372441441448995
Test_r2_max = 1. fea_num: 12  2.r2: 0.16977347868287307
Data/BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.006858067034926106
test_rmse: 0.006841852168658236
test_r2: 0.04405561336890632
feature number: 3
train_rmse: 0.006595013404591339
test_rmse: 0.006542715234996273
test_r2: 0.12489277294863346
feature number: 4
train_rmse: 0.006533806474236625
test_rmse: 0.006480614519835661
test_

train_rmse: 0.006248504848691811
test_rmse: 0.006419391687570081
test_r2: 0.15675798565833335
feature number: 10
train_rmse: 0.00624472238626775
test_rmse: 0.006438421418198624
test_r2: 0.1517744925283699
feature number: 11
train_rmse: 0.0062140248246289
test_rmse: 0.0064182700108168485
test_r2: 0.15705940778216104
feature number: 12
train_rmse: 0.00614445017152416
test_rmse: 0.006374630479415817
test_r2: 0.16923657746207071
feature number: 13
train_rmse: 0.006140172213540274
test_rmse: 0.006400903745815406
test_r2: 0.1624590443837814
feature number: 14
train_rmse: 0.006136688594825404
test_rmse: 0.006418747608878226
test_r2: 0.15777376732916884
feature number: 15
train_rmse: 0.006132087976113152
test_rmse: 0.00644122411004173
test_r2: 0.1518689799950629
feature number: 16
train_rmse: 0.006122656771803006
test_rmse: 0.006502974684357225
test_r2: 0.13539588971964414
feature number: 17
train_rmse: 0.006112094265121483
test_rmse: 0.006522679290807433
test_r2: 0.1300048284660074
feature nu

# 可以修改第一層的神經元數量

In [12]:
#
def build_model():
    model = models.Sequential()
#
    model.add(layers.Dense(6, activation='relu',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [9]:
fea_num=[5,6,7,8,9,10] # 要測試的feature數量 神經元6

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('Data/BayesianRidge_Pre_5/Input_C_096.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 5
test_rmse_ave: 0.006855777837239778
[0.006492486731038795, 0.00704716890932593, 0.006871058248800046, 0.0065751248478286076, 0.007005803291356538, 0.0067933918365618035, 0.006098732741011426, 0.0065210874065682115, 0.006483536642007051, 0.008669387717899373]


test_r2_ave: 0.0004525558736818258
[0.047972011390534974, -0.09464624657273046, 0.039428748788643664, 0.16937521361605168, -0.0404094965447197, 0.03715612636655441, 0.1846997701594767, 0.019091424563507098, 0.12909236807898594, -0.48723436110948604]


feature number: 6
test_rmse_ave: 0.007492884944116297
[0.006619147661313896, 0.006115946149509202, 0.006525818407343243, 0.00766007094365078, 0.0076014402770478605, 0.006702939535896058, 0.00687150943796236, 0.01201030516153229, 0.007989794000846453, 0.006831877866060835]


test_r2_ave: -0.257855468184352
[0.01046373745634821, 0.17553592609769153, 0.13353246871181046, -0.12735958897842337, -0.22484238064731432, 0.06262547814827635, -0.035005494256646674, -2.3273353